In [ ]:
! pip install gTTS SpeechRecognition wikipedia youtube-search-python geopy


In [ ]:
import os
import random
import webbrowser
import datetime
import speech_recognition as sr
from gtts import gTTS
import wikipedia
from youtube_search import YoutubeSearch
from googlesearch import search
from geopy.geocoders import Nominatim

class Assistant:
    def __init__(self):
        self.geolocator = Nominatim(user_agent="assistente-virtual")
        self.greetings = ["Olá, como posso ajudar?", "Oi, estou aqui para ajudar você.", "Olá! Em que posso ser útil?"]
        self.farewells = ["Até logo!", "Até a próxima!", "Até mais!"]
    
    def text_to_speech(self, text):
        tts = gTTS(text=text, lang='pt')
        tts.save('output.mp3')
        os.system('mpg321 output.mp3')  # Reproduzir o áudio
    
    def speech_to_text(self):
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Estou ouvindo, diga algo:")
            audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language='pt-BR')
            print("Você disse:", text)
            return text.lower()
        except sr.UnknownValueError:
            print("Não foi possível entender a fala")
            return ""
        except sr.RequestError as e:
            print("Erro ao acessar o serviço de reconhecimento de fala; {0}".format(e))
            return ""
    
    def search_wikipedia(self, query):
        try:
            result = wikipedia.summary(query, sentences=2)
            print(result)
            self.text_to_speech(result)
        except Exception as e:
            print("Erro ao pesquisar no Wikipedia:", e)
    
    def open_youtube(self, query):
        try:
            results = YoutubeSearch(query, max_results=1).to_dict()
            if results:
                video_id = results[0]['id']
                youtube_url = f"https://www.youtube.com/watch?v={video_id}"
                print("Abrindo vídeo no YouTube:", youtube_url)
                self.text_to_speech("Abrindo vídeo no YouTube")
                webbrowser.open(youtube_url)
            else:
                print("Nenhum resultado encontrado no YouTube")
        except Exception as e:
            print("Erro ao abrir o YouTube:", e)
    
    def search_web(self, query):
        try:
            search_results = search(query, num=5, stop=5, lang='pt')
            for result in search_results:
                print(result)
            self.text_to_speech("Aqui estão os resultados da pesquisa")
        except Exception as e:
            print("Erro ao buscar na web:", e)
    
    def find_nearest_pharmacy(self):
        try:
            location = self.geolocator.geocode("farmácia")
            print("A farmácia mais próxima está localizada em:", location.address)
            self.text_to_speech("A farmácia mais próxima está localizada em " + location.address)
        except Exception as e:
            print("Erro ao encontrar farmácia mais próxima:", e)
    
    def control_assistant(self, command):
        if "parar" in command:
            self.text_to_speech(random.choice(self.farewells))
            return True
        return False
    
    def greet_user(self):
        self.text_to_speech(random.choice(self.greetings))
    
    def main(self):
        self.greet_user()
        while True:
            command = self.speech_to_text()
            if self.control_assistant(command):
                break
            if "wikipedia" in command:
                query = command.replace("wikipedia", "").strip()
                self.search_wikipedia(query)
            elif "youtube" in command:
                query = command.replace("youtube", "").strip()
                self.open_youtube(query)
            elif "farmácia" in command:
                self.find_nearest_pharmacy()
            elif "pesquisar" in command:
                query = command.replace("pesquisar", "").strip()
                self.search_web(query)

if __name__ == "__main__":
    assistant = Assistant()
    assistant.main()
